In [17]:
import pyspark
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")


# Queries:
- the percentage of canceled flights per day, throughout the entire data set
- weekly percentages of delays that are due to weather, throughout the entire data set 
- the percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles.
- a weekly "penalty" score for each airport that depends on both the its incoming and outgoing flights. The score adds 0.5 for each incoming flight that is more than 15 minutes late, and 1 for each outgoing flight that is more than 15 minutes late.


# "The percentage of canceled flights per day, throughout the entire data set"
## Sql version

In [6]:
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('BDdata/1994.csv')

In [86]:
d.count()

5180048

In [87]:
d_ymdc= d.select(d["Year"],d["Month"],d["DayofMonth"],d["Cancelled"])
d_cancelled = d_ymdc.filter(d["Cancelled"]==1). \
    groupBy(d["Year"],d["Month"],d["DayofMonth"]). \
    count().withColumnRenamed("count","cancelled")
d_cancelled.show()

+----+-----+----------+---------+
|Year|Month|DayofMonth|cancelled|
+----+-----+----------+---------+
|1994|   10|        28|       37|
|1994|   12|        26|      159|
|1994|    2|        11|     3649|
|1994|   12|        10|       80|
|1994|    4|        13|      368|
|1994|    6|        23|       88|
|1994|    9|        26|       59|
|1994|   12|        11|       76|
|1994|   11|        24|       14|
|1994|   12|         4|       80|
|1994|    1|        13|      315|
|1994|    4|        28|      124|
|1994|    4|        24|       22|
|1994|    5|        12|       72|
|1994|    8|         8|       43|
|1994|    4|        29|      107|
|1994|    4|        16|       65|
|1994|   10|        20|      212|
|1994|    1|         8|     1206|
|1994|   12|        29|       54|
+----+-----+----------+---------+
only showing top 20 rows



In [88]:
d_total = d_ymdc. \
    groupBy(d["Year"],d["Month"],d["DayofMonth"]). \
    count().withColumnRenamed("count","total")
d_total.show()

+----+-----+----------+-----+
|Year|Month|DayofMonth|total|
+----+-----+----------+-----+
|1994|   10|        28|14847|
|1994|   12|        26|14751|
|1994|    2|        11|14242|
|1994|   12|        10|13409|
|1994|    4|        13|14450|
|1994|    6|        23|14527|
|1994|    9|        26|14680|
|1994|   12|        11|14158|
|1994|   11|        24|11524|
|1994|   12|         4|14088|
|1994|    1|        13|14062|
|1994|    4|        24|13661|
|1994|    4|        28|14364|
|1994|    5|        12|14462|
|1994|    8|         8|14819|
|1994|    4|        29|14226|
|1994|    4|        16|12845|
|1994|   10|        20|14800|
|1994|    1|         8|12605|
|1994|   12|        29|14922|
+----+-----+----------+-----+
only showing top 20 rows



In [89]:
res = d_cancelled.join(d_total, on=[d_cancelled.Year == d_total.Year, d_cancelled.Month == d_total.Month,d_cancelled.DayofMonth == d_total.DayofMonth])\
    .select(d_cancelled.Year,d_cancelled.Month,d_cancelled.DayofMonth,d_cancelled["cancelled"],d_total["total"])
res.show()

+----+-----+----------+---------+-----+
|Year|Month|DayofMonth|cancelled|total|
+----+-----+----------+---------+-----+
|1994|   10|        28|       37|14847|
|1994|   12|        26|      159|14751|
|1994|    2|        11|     3649|14242|
|1994|   12|        10|       80|13409|
|1994|    4|        13|      368|14450|
|1994|    6|        23|       88|14527|
|1994|    9|        26|       59|14680|
|1994|   12|        11|       76|14158|
|1994|   11|        24|       14|11524|
|1994|   12|         4|       80|14088|
|1994|    1|        13|      315|14062|
|1994|    4|        24|       22|13661|
|1994|    4|        28|      124|14364|
|1994|    5|        12|       72|14462|
|1994|    8|         8|       43|14819|
|1994|    4|        29|      107|14226|
|1994|    4|        16|       65|12845|
|1994|   10|        20|      212|14800|
|1994|    1|         8|     1206|12605|
|1994|   12|        29|       54|14922|
+----+-----+----------+---------+-----+
only showing top 20 rows



In [90]:
res.withColumn("percentageCancelled", (res.cancelled/res.total))\
    .drop("cancelled","total")\
    .orderBy("Year","Month","DayofMonth").show()

+----+-----+----------+--------------------+
|Year|Month|DayofMonth| percentageCancelled|
+----+-----+----------+--------------------+
|1994|    1|         1|0.005264023688106...|
|1994|    1|         2|0.004492230650268797|
|1994|    1|         3| 0.01541819205857505|
|1994|    1|         4| 0.15560882746950574|
|1994|    1|         5|0.047656139357031585|
|1994|    1|         6| 0.05416755640970888|
|1994|    1|         7| 0.08754974417282547|
|1994|    1|         8| 0.09567631892106307|
|1994|    1|         9|0.011112791049289385|
|1994|    1|        10|0.018481801444554597|
|1994|    1|        11|0.013738403795765172|
|1994|    1|        12| 0.07870958633347451|
|1994|    1|        13|0.022400796472763475|
|1994|    1|        14| 0.01504349066020248|
|1994|    1|        15|  0.0120414673046252|
|1994|    1|        16|0.047875201721355565|
|1994|    1|        17| 0.17112681081852565|
|1994|    1|        18| 0.11705949985628054|
|1994|    1|        19| 0.07063008130081301|
|1994|    

In [102]:
def getCancelledPercentage(file):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
    .load(file)
    
    d_ymdc= d.select(d["Year"],d["Month"],d["DayofMonth"],d["Cancelled"])
    
    d_cancelled = d_ymdc.filter(d["Cancelled"]==1) \
        .groupBy(d["Year"],d["Month"],d["DayofMonth"]) \
        .count().withColumnRenamed("count","cancelled")
    
    d_total = d_ymdc \
        .groupBy(d["Year"],d["Month"],d["DayofMonth"]) \
        .count().withColumnRenamed("count","total")
    
    res = d_cancelled.join(d_total, on=[d_cancelled.Year == d_total.Year, d_cancelled.Month == d_total.Month,d_cancelled.DayofMonth == d_total.DayofMonth]) \
        .select(d_cancelled.Year,d_cancelled.Month,d_cancelled.DayofMonth,d_cancelled["cancelled"],d_total["total"])
    
    return res.withColumn("percentageCancelled", (res.cancelled/res.total))\
        .drop("cancelled","total")\
        .orderBy("Year","Month","DayofMonth")
        

res1=[getCancelledPercentage("BDdata/"+str(i)+".csv") for i in range(1994,2009)]


In [104]:
results[1].show()

+----+-----+----------+--------------------+
|Year|Month|DayofMonth| percentageCancelled|
+----+-----+----------+--------------------+
|1995|    1|         1|0.010582010582010581|
|1995|    1|         2|0.010852713178294573|
|1995|    1|         3|0.017752621084453593|
|1995|    1|         4| 0.02272874023374443|
|1995|    1|         5| 0.02155618850336613|
|1995|    1|         6| 0.06094487171201448|
|1995|    1|         7|  0.0257183908045977|
|1995|    1|         8| 0.01808698008399946|
|1995|    1|         9|  0.0530588388102351|
|1995|    1|        10| 0.04393730736441734|
|1995|    1|        11| 0.12992796332678455|
|1995|    1|        12| 0.09618030531350324|
|1995|    1|        13|  0.0287468966418398|
|1995|    1|        14|0.027137140778464897|
|1995|    1|        15|0.019543303846945075|
|1995|    1|        16|0.020290613954706112|
|1995|    1|        17|0.016901408450704224|
|1995|    1|        18|0.031135291033036184|
|1995|    1|        19| 0.04605004585353072|
|1995|    

## MapReduce version

In [21]:
ds = sc.textFile('./BDdata/1994.csv')

In [36]:
ds.take(1)

['Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay']

In [39]:
dsplitted = ds.map(lambda line : line.split(","))

head = dsplitted.take(1)[0]
dsfiltered = dsplitted.filter(lambda x : x != head)

dmapped = dsfiltered.map(lambda x : (tuple([int(el) for el in x[0:3]]),int(x[21])))

date_cancelled = dmapped.reduceByKey(lambda a, b : a+b)
total_per_date = dmapped.map(lambda x: (x[0],1)).reduceByKey(lambda a, b : a+b)

results=date_cancelled.join(total_per_date).map(lambda x : (x[0], x[1][0]/x[1][1]))
results.take(20)

[((1994, 1, 5), 0.047656139357031585),
 ((1994, 2, 1), 0.006602768903088392),
 ((1994, 2, 2), 0.005858685677984047),
 ((1994, 2, 3), 0.007247906551263106),
 ((1994, 5, 18), 0.0035822540644805732),
 ((1994, 5, 19), 0.002082176568573015),
 ((1994, 5, 21), 0.003274942878903275),
 ((1994, 5, 16), 0.005455047645352852),
 ((1994, 6, 25), 0.009863996413092213),
 ((1994, 6, 26), 0.003399638336347197),
 ((1994, 6, 27), 0.0069387194284144),
 ((1994, 6, 28), 0.003035738926452325),
 ((1994, 7, 12), 0.008371333287960253),
 ((1994, 7, 13), 0.007550506768247058),
 ((1994, 7, 14), 0.017633442265795208),
 ((1994, 8, 8), 0.0029016802753222214),
 ((1994, 8, 9), 0.0019498419955624286),
 ((1994, 8, 11), 0.003987025273685633),
 ((1994, 8, 14), 0.014527673446247608),
 ((1994, 11, 23), 0.0013551053594416966)]

In [41]:
def getPercentage(year):
    ds = sc.textFile('./BDdata/'+year+'.csv')
    dsplitted = ds.map(lambda line : line.split(","))

    head = dsplitted.take(1)[0]
    dsfiltered = dsplitted.filter(lambda x : x != head)

    dmapped = dsfiltered.map(lambda x : (tuple([int(el) for el in x[0:3]]),int(x[21])))

    date_cancelled = dmapped.reduceByKey(lambda a, b : a+b)
    total_per_date = dmapped.map(lambda x: (x[0],1)).reduceByKey(lambda a, b : a+b)

    results=date_cancelled.join(total_per_date).map(lambda x : (x[0], x[1][0]/x[1][1]))
    return results


In [67]:
res2 = [getPercentage(str(i)).collect() for i in range(1994,2009)]

## Extra query: Weekly percentages of flights cancelled that are due to weather, throughout the entire data set 

We load the data from the file:

In [276]:
d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
    .load('BDdata/2003.csv')


We filter only the flight that have been cancelled and select only the necessary columns:

In [277]:
dr = d.filter(d["Cancelled"]=='1')\
    .select(d["Year"],d["Month"],d["DayofMonth"],d["CancellationCode"])

We then filter only the flights that have been cancelled due to the weather conditions:

In [278]:
d_cancelled_weather = dr.filter(d["CancellationCode"]=='B')
d_cancelled_weather.show()

+----+-----+----------+----------------+
|Year|Month|DayofMonth|CancellationCode|
+----+-----+----------+----------------+
|2003|    6|         1|               B|
|2003|    6|        20|               B|
|2003|    6|        12|               B|
|2003|    6|        12|               B|
|2003|    6|        12|               B|
|2003|    6|        13|               B|
|2003|    6|        25|               B|
|2003|    6|        13|               B|
|2003|    6|        12|               B|
|2003|    6|        13|               B|
|2003|    6|        27|               B|
|2003|    6|        12|               B|
|2003|    6|        12|               B|
|2003|    6|        13|               B|
|2003|    6|        13|               B|
|2003|    6|        27|               B|
|2003|    6|        23|               B|
|2003|    6|        20|               B|
|2003|    6|        12|               B|
|2003|    6|        20|               B|
+----+-----+----------+----------------+
only showing top

Count the total ammount of cancelled flights per day:

In [279]:
d2 = dr.groupBy(d["Year"],d["Month"],d["DayofMonth"]).count().withColumnRenamed("count","cancelled")
d2.show()

+----+-----+----------+---------+
|Year|Month|DayofMonth|cancelled|
+----+-----+----------+---------+
|2003|    1|        19|      184|
|2003|    7|        29|      172|
|2003|   12|         5|     1274|
|2003|   11|         7|      107|
|2003|    3|        26|      187|
|2003|    7|        17|      286|
|2003|   10|        25|      115|
|2003|    5|        12|      154|
|2003|    6|        30|      137|
|2003|    6|        15|       84|
|2003|   11|         5|      423|
|2003|    1|        22|      342|
|2003|    2|         8|       96|
|2003|    2|        10|      532|
|2003|    2|        19|      218|
|2003|    6|         9|      100|
|2003|    8|        15|     1118|
|2003|    8|         9|       71|
|2003|    7|        10|      380|
|2003|    5|        19|      125|
+----+-----+----------+---------+
only showing top 20 rows



We then notice that no data is available before June 2003:

In [280]:
d3 = d_cancelled_weather.groupBy(d["Year"],d["Month"],d["DayofMonth"])\
        .count()\
        .withColumnRenamed("count","weather")

d3.orderBy(d2["Year"],d2["Month"],d2["DayofMonth"]).show()

+----+-----+----------+-------+
|Year|Month|DayofMonth|weather|
+----+-----+----------+-------+
|2003|    6|         1|      8|
|2003|    6|         2|      4|
|2003|    6|         3|     15|
|2003|    6|         4|     23|
|2003|    6|         5|     46|
|2003|    6|         6|      5|
|2003|    6|         7|      4|
|2003|    6|         8|     24|
|2003|    6|         9|      8|
|2003|    6|        10|      7|
|2003|    6|        11|     20|
|2003|    6|        12|    191|
|2003|    6|        13|    175|
|2003|    6|        14|     44|
|2003|    6|        15|      5|
|2003|    6|        16|     44|
|2003|    6|        17|      8|
|2003|    6|        18|     38|
|2003|    6|        19|     13|
|2003|    6|        20|     11|
+----+-----+----------+-------+
only showing top 20 rows



We join the data of the total cancelled flights and the ones cancelled due to weather in a single table, ordering it afterwards to better visualize the data:

In [281]:
d4 = d2.join(d3, on=[d2["Year"]==d3["Year"],d2["Month"]==d3["Month"],d2["DayofMonth"]==d3["DayofMonth"]])\
    .select(d2["Year"],d2["Month"],d2["DayofMonth"],d2["cancelled"],d3["weather"])\
    .orderBy(d2["Year"],d2["Month"],d2["DayofMonth"])
d4.show()

+----+-----+----------+---------+-------+
|Year|Month|DayofMonth|cancelled|weather|
+----+-----+----------+---------+-------+
|2003|    6|         1|       73|      8|
|2003|    6|         2|       94|      4|
|2003|    6|         3|      119|     15|
|2003|    6|         4|      148|     23|
|2003|    6|         5|      178|     46|
|2003|    6|         6|      102|      5|
|2003|    6|         7|       63|      4|
|2003|    6|         8|      119|     24|
|2003|    6|         9|      100|      8|
|2003|    6|        10|      124|      7|
|2003|    6|        11|      143|     20|
|2003|    6|        12|      526|    191|
|2003|    6|        13|      385|    175|
|2003|    6|        14|      143|     44|
|2003|    6|        15|       84|      5|
|2003|    6|        16|      182|     44|
|2003|    6|        17|      129|      8|
|2003|    6|        18|      225|     38|
|2003|    6|        19|      102|     13|
|2003|    6|        20|      102|     11|
+----+-----+----------+---------+-

We have to associate to each row the corresponding week number, taking into account the fact the the last days of December counts as first week of the successive year, so we have to remove them before grouping by week:

In [282]:
week = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[1], IntegerType())

d5 = d4.withColumn("week", week(d4["Year"],d4["Month"],d4["DayofMonth"]))

newYearFirstWeek = d5.filter(d5["week"]==1).filter(d5["Month"]=="12")

d5a = d5.filter((d5["week"]!=1) | (d5["Month"]!="12")).groupBy("Year","week").agg({"cancelled":"sum","weather":"sum"})
d5a.show()

+----+----+------------+--------------+
|Year|week|sum(weather)|sum(cancelled)|
+----+----+------------+--------------+
|2003|  50|        1324|          3212|
|2003|  32|         339|          1654|
|2003|  35|         462|          1755|
|2003|  46|         677|          2119|
|2003|  40|         107|           889|
|2003|  22|           8|            73|
|2003|  49|        3527|          5135|
|2003|  25|         119|           877|
|2003|  28|         804|          2662|
|2003|  26|          79|           746|
|2003|  37|         203|          1489|
|2003|  51|         326|          1671|
|2003|  33|         293|          2942|
|2003|  36|         114|          1160|
|2003|  27|         361|          1101|
|2003|  23|         121|           823|
|2003|  47|         952|          2348|
|2003|  44|         206|          1547|
|2003|  29|         255|          1302|
|2003|  41|         190|           760|
+----+----+------------+--------------+
only showing top 20 rows



In [283]:
newYearFirstWeek.show()

+----+-----+----------+---------+-------+----+
|Year|Month|DayofMonth|cancelled|weather|week|
+----+-----+----------+---------+-------+----+
|2003|   12|        29|      184|     43|   1|
|2003|   12|        30|      132|      6|   1|
|2003|   12|        31|       93|     18|   1|
+----+-----+----------+---------+-------+----+



We then compute the percentage of then sort them to better visualize them:

In [284]:
d6 = d5a.withColumn("percentage",d5a["sum(weather)"]/d5a["sum(cancelled)"]).drop("sum(weather)","sum(cancelled)")\
    .sort("Year","week")
d6.show()

+----+----+-------------------+
|Year|week|         percentage|
+----+----+-------------------+
|2003|  22| 0.1095890410958904|
|2003|  23|0.14702308626974483|
|2003|  24|0.29900332225913623|
|2003|  25|0.13568985176738882|
|2003|  26|0.10589812332439678|
|2003|  27| 0.3278837420526794|
|2003|  28| 0.3020285499624343|
|2003|  29|  0.195852534562212|
|2003|  30| 0.3480414227825304|
|2003|  31| 0.2532659081331648|
|2003|  32|0.20495767835550183|
|2003|  33|0.09959211420802175|
|2003|  34|0.06289308176100629|
|2003|  35|0.26324786324786326|
|2003|  36|0.09827586206896552|
|2003|  37|0.13633310946944258|
|2003|  38| 0.6686946902654868|
|2003|  39| 0.1607773851590106|
|2003|  40| 0.1203599550056243|
|2003|  41|               0.25|
+----+----+-------------------+
only showing top 20 rows



We then check what are the years that have the needed informations to compute the query result:

In [240]:
for i in range(1994,2009):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('BDdata/'+str(i)+'.csv')
    print(i," : ",d.select(d["CancellationCode"]).distinct().rdd.collect())

1994  :  [Row(CancellationCode='NA')]
1995  :  [Row(CancellationCode='NA')]
1996  :  [Row(CancellationCode='NA')]
1997  :  [Row(CancellationCode='NA')]
1998  :  [Row(CancellationCode='NA')]
1999  :  [Row(CancellationCode='NA')]
2000  :  [Row(CancellationCode='NA')]
2001  :  [Row(CancellationCode='NA')]
2002  :  [Row(CancellationCode='NA')]
2003  :  [Row(CancellationCode='NA'), Row(CancellationCode=None), Row(CancellationCode='B'), Row(CancellationCode='D'), Row(CancellationCode='C'), Row(CancellationCode='A')]
2004  :  [Row(CancellationCode=None), Row(CancellationCode='B'), Row(CancellationCode='D'), Row(CancellationCode='C'), Row(CancellationCode='A')]
2005  :  [Row(CancellationCode=None), Row(CancellationCode='B'), Row(CancellationCode='D'), Row(CancellationCode='C'), Row(CancellationCode='A')]
2006  :  [Row(CancellationCode=None), Row(CancellationCode='B'), Row(CancellationCode='D'), Row(CancellationCode='C'), Row(CancellationCode='A')]
2007  :  [Row(CancellationCode=None), Row(Canc

In [285]:
week = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[1], IntegerType())

In [286]:

def getWeatherPercentage(year):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('BDdata/'+year+'.csv')
    dr = d.filter(d["Cancelled"]=='1')\
        .select(d["Year"],d["Month"],d["DayofMonth"],d["CancellationCode"])

    d_cancelled_weather = dr.filter(d["CancellationCode"]=='B')

    d2 = dr.groupBy(d["Year"],d["Month"],d["DayofMonth"])\
        .count()\
        .withColumnRenamed("count","cancelled")
    
    d3 = d_cancelled_weather.groupBy(d["Year"],d["Month"],d["DayofMonth"])\
        .count()\
        .withColumnRenamed("count","weather")

    d3.orderBy(d2["Year"],d2["Month"],d2["DayofMonth"]).show()
    
    d4 = d2.join(d3, on=[d2["Year"]==d3["Year"],d2["Month"]==d3["Month"],d2["DayofMonth"]==d3["DayofMonth"]])\
        .select(d2["Year"],d2["Month"],d2["DayofMonth"],d2["cancelled"],d3["weather"])\
        .orderBy(d2["Year"],d2["Month"],d2["DayofMonth"])
        
    d5 = d4.withColumn("week", week(d4["Year"],d4["Month"],d4["DayofMonth"]))
    
    newYearFirstWeek = d5.filter(d5["week"]==1).filter(d5["Month"]=="12")
    
    d5a = d5.filter((d5["week"]!=1) | (d5["Month"]!="12")).groupBy("Year","week")\
        .agg({"cancelled":"sum","weather":"sum"})
    d6 = d5a.withColumn("percentage",d5a["sum(weather)"]/d5a["sum(cancelled)"])\
        .drop("sum(weather)","sum(cancelled)")\
        .sort("Year","week")


# Weekly percentages of delays that are due to weather, throughout the entire data set 

In [307]:
d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
    .load('BDdata/2003.csv')
d1 = d.where("ArrDelay >= 0").select(d["Year"],d["Month"],d["DayofMonth"],d["ArrDelay"],d["WeatherDelay"])\
    .where("WeatherDelay != 'NA'")
d1.show()

+----+-----+----------+--------+------------+
|Year|Month|DayofMonth|ArrDelay|WeatherDelay|
+----+-----+----------+--------+------------+
|2003|    6|        29|      26|          26|
|2003|    6|         1|       3|           0|
|2003|    6|         2|      14|           0|
|2003|    6|        10|       0|           0|
|2003|    6|         6|       4|           0|
|2003|    6|         7|       0|           0|
|2003|    6|         8|      17|           0|
|2003|    6|         9|       6|           0|
|2003|    6|        10|       1|           0|
|2003|    6|        11|       5|           0|
|2003|    6|        12|      19|           0|
|2003|    6|        13|      24|           0|
|2003|    6|        17|      11|           0|
|2003|    6|        20|      12|           0|
|2003|    6|        21|       0|           0|
|2003|    6|        22|       7|           0|
|2003|    6|        23|      17|           0|
|2003|    6|        27|      21|           0|
|2003|    6|        28|       2|  

In [308]:
d3 = d1.groupBy(d["Year"],d["Month"],d["DayofMonth"]).agg({"ArrDelay":"sum","WeatherDelay":"sum"})
d3.show()

+----+-----+----------+-----------------+-------------+
|Year|Month|DayofMonth|sum(WeatherDelay)|sum(ArrDelay)|
+----+-----+----------+-----------------+-------------+
|2003|    7|        29|           8979.0|     137186.0|
|2003|   12|         5|          40077.0|     501010.0|
|2003|   11|         7|           2961.0|     155884.0|
|2003|    7|        17|          13679.0|     203466.0|
|2003|   10|        25|           5802.0|      83849.0|
|2003|    6|        30|           6734.0|     140096.0|
|2003|    6|        15|           3589.0|     113804.0|
|2003|   11|         5|          22131.0|     285687.0|
|2003|    6|         9|           4347.0|     113139.0|
|2003|    8|         9|          17343.0|     188613.0|
|2003|    8|        15|           5238.0|     209983.0|
|2003|    7|        10|          32711.0|     333060.0|
|2003|    6|        18|          11921.0|     225328.0|
|2003|   10|         1|           1210.0|     112603.0|
|2003|   10|        17|           1378.0|     12

In [309]:
d4 = d3.withColumn("week", week(d3["Year"],d3["Month"],d3["DayofMonth"]))
d4.show()

+----+-----+----------+-----------------+-------------+----+
|Year|Month|DayofMonth|sum(WeatherDelay)|sum(ArrDelay)|week|
+----+-----+----------+-----------------+-------------+----+
|2003|    7|        29|           8979.0|     137186.0|  31|
|2003|   12|         5|          40077.0|     501010.0|  49|
|2003|   11|         7|           2961.0|     155884.0|  45|
|2003|    7|        17|          13679.0|     203466.0|  29|
|2003|   10|        25|           5802.0|      83849.0|  43|
|2003|    6|        30|           6734.0|     140096.0|  27|
|2003|    6|        15|           3589.0|     113804.0|  24|
|2003|   11|         5|          22131.0|     285687.0|  45|
|2003|    6|         9|           4347.0|     113139.0|  24|
|2003|    8|         9|          17343.0|     188613.0|  32|
|2003|    8|        15|           5238.0|     209983.0|  33|
|2003|    7|        10|          32711.0|     333060.0|  28|
|2003|    6|        18|          11921.0|     225328.0|  25|
|2003|   10|         1| 

In [310]:
d4.where("week = 1").show()

+----+-----+----------+-----------------+-------------+----+
|Year|Month|DayofMonth|sum(WeatherDelay)|sum(ArrDelay)|week|
+----+-----+----------+-----------------+-------------+----+
|2003|   12|        31|           2829.0|      90979.0|   1|
|2003|   12|        30|           5751.0|     209404.0|   1|
|2003|   12|        29|           8638.0|     249379.0|   1|
+----+-----+----------+-----------------+-------------+----+



In [311]:
newYearFirstWeek = d4.filter(d4["week"]==1).filter(d4["Month"]=="12")
d5a = d4.filter((d4["week"]!=1) | (d4["Month"]!="12")).groupBy("Year","week")\
        .agg({"sum(WeatherDelay)":"sum","sum(ArrDelay)":"sum"})
d5a.show()

+----+----+----------------------+------------------+
|Year|week|sum(sum(WeatherDelay))|sum(sum(ArrDelay))|
+----+----+----------------------+------------------+
|2003|  50|               98311.0|         1672254.0|
|2003|  32|               90932.0|         1580956.0|
|2003|  35|              108086.0|         1329545.0|
|2003|  46|               52927.0|         1336317.0|
|2003|  40|               14404.0|          624630.0|
|2003|  49|              104118.0|         1624586.0|
|2003|  22|                2266.0|          104139.0|
|2003|  25|               69910.0|         1258701.0|
|2003|  28|              128860.0|         1956895.0|
|2003|  26|               28127.0|         1058286.0|
|2003|  37|               58503.0|         1000855.0|
|2003|  51|               42295.0|         1629182.0|
|2003|  33|               90469.0|         1611533.0|
|2003|  36|               31188.0|          883154.0|
|2003|  27|               73370.0|         1076466.0|
|2003|  23|               51

In [312]:
d5a.where("week = 1").show()

+----+----+----------------------+------------------+
|Year|week|sum(sum(WeatherDelay))|sum(sum(ArrDelay))|
+----+----+----------------------+------------------+
+----+----+----------------------+------------------+



In [313]:
 d6 = d5a.withColumn("percentage",d5a["sum(sum(WeatherDelay))"]/d5a["sum(sum(ArrDelay))"])\
        .drop("sum(sum(WeatherDelay))","sum(sum(ArrDelay))")\
        .sort("Year","week")
    
d6.show()

+----+----+--------------------+
|Year|week|          percentage|
+----+----+--------------------+
|2003|  22|0.021759379291139726|
|2003|  23| 0.05435799236788522|
|2003|  24| 0.08422798224375042|
|2003|  25|0.055541387509821634|
|2003|  26|0.026577881593444493|
|2003|  27| 0.06815821400768812|
|2003|  28| 0.06584921521083144|
|2003|  29| 0.03957742490221589|
|2003|  30| 0.07160054816598709|
|2003|  31| 0.07311418046852729|
|2003|  32| 0.05751709725001834|
|2003|  33|0.056138471877398725|
|2003|  34|0.060240938882279056|
|2003|  35| 0.08129548078477976|
|2003|  36| 0.03531433928850461|
|2003|  37| 0.05845302266562089|
|2003|  38|0.036452499395229015|
|2003|  39|   0.036826458565589|
|2003|  40|0.023060051550517907|
|2003|  41| 0.04255098964109221|
+----+----+--------------------+
only showing top 20 rows



In [340]:
def getWeatherDelayPercentage(year):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('BDdata/'+year+'.csv')

    d1 = d.where("ArrDelay >= 0").select(d["Year"],d["Month"],d["DayofMonth"],d["ArrDelay"],d["WeatherDelay"])
    if d1.where("WeatherDelay == 'NA'").count() > 0:
        d1= d1.where("WeatherDelay != 'NA'")

    d3 = d1.groupBy(d["Year"],d["Month"],d["DayofMonth"]).agg({"ArrDelay":"sum","WeatherDelay":"sum"})
    
    week = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[1], IntegerType())

    d4 = d3.withColumn("week", week(d3["Year"],d3["Month"],d3["DayofMonth"]))

    newYearFirstWeek = d4.filter(d4["week"]==1).filter(d4["Month"]=="12")
    
    d5a = d4.filter((d4["week"]!=1) | (d4["Month"]!="12")).groupBy("Year","week")\
            .agg({"sum(WeatherDelay)":"sum","sum(ArrDelay)":"sum"})

    d6 = d5a.withColumn("percentage",d5a["sum(sum(WeatherDelay))"]/d5a["sum(sum(ArrDelay))"])\
            .drop("sum(sum(WeatherDelay))","sum(sum(ArrDelay))")\
            .sort("Year","week")
    
    return d6

getWeatherDelayPercentage("2005").show()

+----+----+--------------------+
|Year|week|          percentage|
+----+----+--------------------+
|2005|   1| 0.06666523925107397|
|2005|   2|0.061525768148909366|
|2005|   3| 0.09819828459897512|
|2005|   4| 0.07318699654293981|
|2005|   5|0.059825071738306534|
|2005|   6|0.042730355121855995|
|2005|   7|   0.042275550868367|
|2005|   8|  0.0526360788579327|
|2005|   9| 0.04082060164623675|
|2005|  10| 0.05587787669915344|
|2005|  11| 0.04586946314758946|
|2005|  12|0.045018706367879045|
|2005|  13| 0.05549382003485314|
|2005|  14| 0.03788709568855268|
|2005|  15|  0.0315842412528014|
|2005|  16| 0.05604493024466552|
|2005|  17| 0.05655818168088802|
|2005|  18| 0.04442350982424638|
|2005|  19| 0.04078598684307204|
|2005|  20| 0.05272002108407274|
+----+----+--------------------+
only showing top 20 rows



## The percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. 

Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles.

In [360]:
d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('BDdata/2005.csv')
d1 = d.where(d["Cancelled"]==0).select(d["DepDelay"],d["ArrDelay"],d["Distance"])
d2 = d1.where(d["DepDelay"]>0)
distanceGroup = udf(lambda x : x//200+1,IntegerType())
d3 = d2.filter(d2["DepDelay"]/d2["ArrDelay"]>=2).withColumn("DistanceGroup",distanceGroup(d2["Distance"]))\
    .drop(d2["Distance"])
d4 = d3.groupBy(d3.DistanceGroup).agg({"*":"count"})
d5 = d2.withColumn("DistanceGroup",distanceGroup(d2["Distance"]))\
    .drop(d2["Distance"])
d6 = d5.groupBy(d5.DistanceGroup).agg({"*":"count"})
d6.show()


+-------------+--------+
|DistanceGroup|count(1)|
+-------------+--------+
|           12|   33528|
|           22|     592|
|            1|  198627|
|           13|   50040|
|            6|  200053|
|           16|     134|
|            3|  461810|
|           20|    1079|
|            5|  292713|
|           19|     668|
|           15|    1934|
|            9|   70000|
|           17|     504|
|            4|  340342|
|            8|  101648|
|           23|     651|
|            7|   85147|
|           10|   52966|
|           25|     274|
|           21|     443|
+-------------+--------+
only showing top 20 rows



In [361]:
d7 = d4.join(d6,on=[d4.DistanceGroup==d6.DistanceGroup])\
    .withColumn("percentage", d4["count(1)"]/d6["count(1)"]).drop("count(1)")
d8 = d7.select(d4.DistanceGroup,d7.percentage).sort("DistanceGroup")
d8.show()

+-------------+-------------------+
|DistanceGroup|         percentage|
+-------------+-------------------+
|            1|0.08827601484188957|
|            2|0.09693922686977234|
|            3|0.09701175808232823|
|            4| 0.1027672165057501|
|            5|0.10443335280633248|
|            6|0.10570198897292217|
|            7| 0.1098923038979647|
|            8|0.10642609790650087|
|            9|0.10832857142857143|
|           10|0.10873012876184722|
|           11|0.10228890592487132|
|           12| 0.1111608208064901|
|           13|0.09840127897681855|
|           14|0.10471845092365903|
|           15|0.10237849017580145|
|           16| 0.1044776119402985|
|           17|0.08134920634920635|
|           18|0.04950495049504951|
|           19|0.08982035928143713|
|           20|0.10843373493975904|
+-------------+-------------------+
only showing top 20 rows



## A weekly "penalty" score for each airport that depends on both the its incoming and outgoing flights. The score adds 0.5 for each incoming flight that is more than 15 minutes late, and 1 for each outgoing flight that is more than 15 minutes late.